# PySpark Tutorial - Joins
<div>
 <h2> CSCI 4253 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 36.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f95cc6f0d2f6e9fa49d2969e542d6fb4d835312f0182827aaac9d4421b528f4b
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf
import numpy as np
import operator

In [3]:
conf=SparkConf().setAppName("pyspark tutorial").setMaster("local[*]")
sc = SparkContext(conf=conf)

## Key Value Data, Grouping and Joins

The Key-Value or (k,v) datatype is fundemental to many operations including grouping and joins. We need to be able to:
* Create keys from non-KV data
* Group or organize data according to keys
* Operate on data according to keys
* Form standard joins

### Creating Key-Pair Values

We can directly create K-V pairs using lists of pairs

In [8]:
visits = sc.parallelize([ ("index.html", "1.2.3.4"),
                         ("about.html", "3.4.5.6"),
                         ("index.html", "1.3.3.1"),
                        ("index.html", "2.2.2.2")])

pageNames = sc.parallelize([ ("index.html", "Home"),
                            ("about.html", "About") ])

When we `collect()` the K-V pairs, we'll get the full list of keys -- this pulls the data to the front-end machine:

In [9]:
visits.collect()

[('index.html', '1.2.3.4'),
 ('about.html', '3.4.5.6'),
 ('index.html', '1.3.3.1'),
 ('index.html', '2.2.2.2')]

The first position of the K-V pair is the key. We'll cover grouping in more detail later, but let's take a look at grouping visits by the keys:

In [10]:
visits.groupByKey().collect()

[('about.html', <pyspark.resultiterable.ResultIterable at 0x7fc7b08cb190>),
 ('index.html', <pyspark.resultiterable.ResultIterable at 0x7fc7b08cb1d0>)]

The KV pairs have now been *grouped* meaning that we have an RDD of all the keys and the values for each key are a "list" of of the values corresponding to that key in the original KV RDD. Because the values are scattered across your cluster, a `ResultIterable` is used to represented their distributed type.

We reify the results by converting it to a list. You wouldn't do this in practice because this brings all the values back to the front-end machine across the whole networking, but lets see what this produces to understand what `groupByKey` is doing.

We'll `map` a lambda function that simply "flattens" the items by converting the `ResultIterable` into a list.

In [13]:
visits.groupByKey().map(lambda x: (x[0], list(x[1]))).collect()

[('about.html', ['3.4.5.6']),
 ('index.html', ['1.2.3.4', '1.3.3.1', '2.2.2.2'])]

When we `map` across a grouped KV the mapped function takes an argument which is the pair "(key, value") -- that's why we're referring to `x[0]` for the value and `x[1]` for the value in the sample `map`.

It's more typicaly that you want to just `map` across the values and there is a corresponding `mapValues` function that does precisely this. In the example below, we are mapping `list` across the values in the groups. The results should be the same as the 

In [14]:
visits.groupByKey().mapValues(list).collect()

[('about.html', ['3.4.5.6']),
 ('index.html', ['1.2.3.4', '1.3.3.1', '2.2.2.2'])]

`keyBy` is a function to efficiently create a key-value pair from an RDD. The elements of the original RDD are the "values" and a function provides the associated key.

For example, assume we want $K = V^2$ for values $V$ in an RDD:

In [15]:
r = sc.parallelize( [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3] )
r.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3]

In [16]:
rsq = r.keyBy(lambda x : x*x)
rsq.collect()

[(0, 0),
 (1, 1),
 (4, 2),
 (9, 3),
 (16, 4),
 (25, 5),
 (36, 6),
 (49, 7),
 (64, 8),
 (81, 9),
 (0, 0),
 (1, 1),
 (4, 2),
 (9, 3)]

For example, we might want to create keys for the **passwd** data using the shell name

In [17]:
passwd = sc.textFile("/etc/passwd")

The password is the 7th field (6th index)  in the `/etc/passwd` file:

In [22]:
passwd.take(1)[0].split(':')

['root', 'x', '0', '0', 'root', '/root', '/bin/bash']

Thus, if we want the `key` to be the 6th element, we provide a function that extracts that from each entry:

In [19]:
byShell = passwd.keyBy( lambda x : x.split(':')[6] )
byShell.take(3)

[('/bin/bash', 'root:x:0:0:root:/root:/bin/bash'),
 ('/usr/sbin/nologin', 'daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin'),
 ('/usr/sbin/nologin', 'bin:x:2:2:bin:/bin:/usr/sbin/nologin')]

This is more or less equivilent to a `map` that returns pairs of values. For example:

In [23]:
passwd.map( lambda x : (x.split(':')[6], x) ).take(3)

[('/bin/bash', 'root:x:0:0:root:/root:/bin/bash'),
 ('/usr/sbin/nologin', 'daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin'),
 ('/usr/sbin/nologin', 'bin:x:2:2:bin:/bin:/usr/sbin/nologin')]

### Using Key-Value Pairs

We can return a dictionary containing the number of logins using each shell using the `countByKey` function:

In [24]:
byShell.countByKey()

defaultdict(int, {'/bin/bash': 1, '/usr/sbin/nologin': 21, '/bin/sync': 1})

And we can extract keys and values in parallel across the cluster:

In [25]:
shellKeys = byShell.keys()
shellKeys.take(3)

['/bin/bash', '/usr/sbin/nologin', '/usr/sbin/nologin']

In [26]:
byShell.values().take(3)

['root:x:0:0:root:/root:/bin/bash',
 'daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin',
 'bin:x:2:2:bin:/bin:/usr/sbin/nologin']

## Grouping and Joins

Once you have (k,v) pairs, you can group the key -- the values are iterable ( distributed lists) of the values for that key. Earlier, we shows that you can `map` a function over the `ResultIterable` -- it's unlikely you want want to return them to a native `list` because that will pull all the values back to the front end machine.

In [27]:
byShell.groupByKey().take(3)

[('/bin/bash', <pyspark.resultiterable.ResultIterable at 0x7fc7b12f3410>),
 ('/usr/sbin/nologin',
  <pyspark.resultiterable.ResultIterable at 0x7fc7b07f4890>),
 ('/bin/sync', <pyspark.resultiterable.ResultIterable at 0x7fc7b12f3b90>)]

We can also group KV pairs by other attributes using the groupBy() method. For example, here we're going to group the password data by the user name (the 0'th field of the values)

In [28]:
byShell.take(3)

[('/bin/bash', 'root:x:0:0:root:/root:/bin/bash'),
 ('/usr/sbin/nologin', 'daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin'),
 ('/usr/sbin/nologin', 'bin:x:2:2:bin:/bin:/usr/sbin/nologin')]

In [29]:
def getLogin(x):
    return x.split(':')[0]

In [38]:
byShell.map(lambda x : getLogin(x[1])).take(3)

['root', 'daemon', 'bin']

In [30]:
byShell.groupBy (lambda x : getLogin(x[1]) ).take(3)

[('root', <pyspark.resultiterable.ResultIterable at 0x7fc7a8665690>),
 ('daemon', <pyspark.resultiterable.ResultIterable at 0x7fc7a8665190>),
 ('sync', <pyspark.resultiterable.ResultIterable at 0x7fc7a8665550>)]

Once items are grouped, you can iterate over the values associated with a key.

Let's group the `/etc/passwd` entries by the shell.

In [39]:
grpdShell = byShell.groupByKey()
grpdShell.take(3)

[('/bin/bash', <pyspark.resultiterable.ResultIterable at 0x7fc7a8ca2d90>),
 ('/usr/sbin/nologin',
  <pyspark.resultiterable.ResultIterable at 0x7fc7b1356f10>),
 ('/bin/sync', <pyspark.resultiterable.ResultIterable at 0x7fc7b07f4510>)]

Now the key are differ Unix shells (*e.g.* `/bin/bash`) and the values are `ResultIterable`s of the values having that key in the original KV list.

Again, we typically wouldn't want to pull in all the value using a `list` because this will bring everything to the front-end machine in the cluster. But, let's convert the `ResultIterable` to a list just to see the structure:

In [40]:
grpdShell.map(lambda x: (x[0], list(x[1])) ).take(3)

[('/bin/bash', ['root:x:0:0:root:/root:/bin/bash']),
 ('/usr/sbin/nologin',
  ['daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin',
   'bin:x:2:2:bin:/bin:/usr/sbin/nologin',
   'sys:x:3:3:sys:/dev:/usr/sbin/nologin',
   'games:x:5:60:games:/usr/games:/usr/sbin/nologin',
   'man:x:6:12:man:/var/cache/man:/usr/sbin/nologin',
   'lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin',
   'mail:x:8:8:mail:/var/mail:/usr/sbin/nologin',
   'news:x:9:9:news:/var/spool/news:/usr/sbin/nologin',
   'uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin',
   'proxy:x:13:13:proxy:/bin:/usr/sbin/nologin',
   'www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin',
   'backup:x:34:34:backup:/var/backups:/usr/sbin/nologin',
   'list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin',
   'irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin',
   'gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin',
   'nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin',
   '_apt:x:100:655

The better way to process this would be to would `map` a function over each KV pair in the grouped list. The key is `x[0]` and the `ResultIterable` is `x[1]`. We can then map a function over each of the  `ResultIterable` to extract some field, such as the login information.

In [47]:
grpdShell.mapValues(list).take(3)

[('/bin/bash', ['root:x:0:0:root:/root:/bin/bash']),
 ('/usr/sbin/nologin',
  ['daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin',
   'bin:x:2:2:bin:/bin:/usr/sbin/nologin',
   'sys:x:3:3:sys:/dev:/usr/sbin/nologin',
   'games:x:5:60:games:/usr/games:/usr/sbin/nologin',
   'man:x:6:12:man:/var/cache/man:/usr/sbin/nologin',
   'lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin',
   'mail:x:8:8:mail:/var/mail:/usr/sbin/nologin',
   'news:x:9:9:news:/var/spool/news:/usr/sbin/nologin',
   'uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin',
   'proxy:x:13:13:proxy:/bin:/usr/sbin/nologin',
   'www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin',
   'backup:x:34:34:backup:/var/backups:/usr/sbin/nologin',
   'list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin',
   'irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin',
   'gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin',
   'nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin',
   '_apt:x:100:655

In [42]:
shellAndLogins = grpdShell.map( 
    lambda x: (x[0], ",".join( [ getLogin(y) for y in x[1] ]) ))

In [43]:
shellAndLogins.take(3)

[('/bin/bash', 'root'),
 ('/usr/sbin/nologin',
  'daemon,bin,sys,games,man,lp,mail,news,uucp,proxy,www-data,backup,list,irc,gnats,nobody,_apt,systemd-network,systemd-resolve,messagebus,nvidia-persistenced'),
 ('/bin/sync', 'sync')]

Rather than first group the keys and then combine the values into the string above, we can use **foldByKey** to do more or less the same thing -- this combines the mapping phase implicit in the list comprehension above.

In [48]:
byShell.foldByKey( "", lambda x,y: x + getLogin(y) + ',' ).collect()

[('/bin/bash', 'root,'),
 ('/usr/sbin/nologin',
  'daemon,bin,sys,games,man,lp,mail,news,uucp,proxy,www-data,backup,list,irc,gnats,nobody,_apt,systemd-network,systemd-resolve,messagebus,nvidia-persistenced,,'),
 ('/bin/sync', 'sync,')]

Internally, this is done using a "combiner(createCombiner, mergeValue, mergeCombiners)", which turns an RDD[(K, V)] into a result of type RDD[(K, C)], for a "combined type" C.  Note that V and C can be different -- for example, one might group an RDD of type (Int, Int) into an RDD of type (Int, List[Int]).

This example takes the byShell (K,V) list and constructs a new V that is the name of the users of that shell. Entries within the same RDD partition are joined by a comma and between partitions by "AND".

In [49]:
byShell.combineByKey( getLogin,
                        lambda xs, x: xs + ',' + getLogin(x),
                        lambda xs, ys: xs + ' AND ' + ys ).collect()

[('/bin/bash', 'root'),
 ('/usr/sbin/nologin',
  'daemon,bin,sys,games,man,lp,mail,news,uucp,proxy,www-data,backup AND list,irc,gnats,nobody,_apt,systemd-network,systemd-resolve,messagebus,nvidia-persistenced'),
 ('/bin/sync', 'sync')]

**combineByKey** is used to develop "reduceByKey" functions that can e.g. sum up the items associated with a key. This is effectively doing a **groupByKey** followed by a **reduce** on each list of values

In [50]:
c = sc.parallelize([ (1,2), (2,3), (1, 99), (3, 44), (2, 1), (4,5), (3, 19) ] )

The `groupByKey` operator gives us `iterable` items.

In [51]:
c.groupByKey().collect()

[(2, <pyspark.resultiterable.ResultIterable at 0x7fc7a805c250>),
 (4, <pyspark.resultiterable.ResultIterable at 0x7fc7a805c810>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7fc7a805c750>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7fc7a805c890>)]

We can reify (make concrete) the iterable item by convert it to a list:

In [52]:
c.groupByKey().map(lambda x : (x[0], list(x[1]))).collect()

[(2, [3, 1]), (4, [5]), (1, [2, 99]), (3, [44, 19])]

The `reduceByKey` groups identical keys and then applys a function over the iterable results.

In [53]:
c.reduceByKey( operator.add ).collect()

[(2, 4), (4, 5), (1, 101), (3, 63)]

## CoGroup - the basis for joins

As in Pig, joins are done performing "co-groups" where multiple data sets are grouped by the same key

In [54]:
s1 = c
s1.collect()

[(1, 2), (2, 3), (1, 99), (3, 44), (2, 1), (4, 5), (3, 19)]

In [55]:
s2 = sc.parallelize( [ (2, -99), ( 4, 199), (19, 23) ] )
s2.collect()

[(2, -99), (4, 199), (19, 23)]

In [56]:
co = s1.cogroup(s2)
co.collect()

[(4,
  (<pyspark.resultiterable.ResultIterable at 0x7fc7a80e6f90>,
   <pyspark.resultiterable.ResultIterable at 0x7fc7a7f89c50>)),
 (1,
  (<pyspark.resultiterable.ResultIterable at 0x7fc7a80e6d50>,
   <pyspark.resultiterable.ResultIterable at 0x7fc7a80e6f50>)),
 (2,
  (<pyspark.resultiterable.ResultIterable at 0x7fc7a80e6b50>,
   <pyspark.resultiterable.ResultIterable at 0x7fc7a80e6990>)),
 (3,
  (<pyspark.resultiterable.ResultIterable at 0x7fc7a80acc10>,
   <pyspark.resultiterable.ResultIterable at 0x7fc7a80ac550>)),
 (19,
  (<pyspark.resultiterable.ResultIterable at 0x7fc7a80acc50>,
   <pyspark.resultiterable.ResultIterable at 0x7fc7a80ac510>))]

Let's use our "convert it to a list trick" to see what's in each cogroup

In [57]:
co.map(lambda x: (x[0], list(x[1][0]), list(x[1][1])) ).collect()

[(4, [5], [199]),
 (1, [2, 99], []),
 (2, [3, 1], [-99]),
 (3, [44, 19], []),
 (19, [], [23])]

This is used to build different kinds of joins

In [65]:
s1.groupByKey().collect()

[(2, <pyspark.resultiterable.ResultIterable at 0x7fc7a805c5d0>),
 (4, <pyspark.resultiterable.ResultIterable at 0x7fc7a805ced0>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7fc7a805c0d0>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7fc7a805ca10>)]

In [63]:
s2.groupByKey().collect()

[(2, <pyspark.resultiterable.ResultIterable at 0x7fc7a7f95450>),
 (4, <pyspark.resultiterable.ResultIterable at 0x7fc7a7f95210>),
 (19, <pyspark.resultiterable.ResultIterable at 0x7fc7a7f7ded0>)]

In [58]:
s1.join(s2).collect()

[(4, (5, 199)), (2, (3, -99)), (2, (1, -99))]

In [59]:
s1.leftOuterJoin(s2).collect()

[(4, (5, 199)),
 (1, (2, None)),
 (1, (99, None)),
 (2, (3, -99)),
 (2, (1, -99)),
 (3, (44, None)),
 (3, (19, None))]

In [60]:
s1.rightOuterJoin(s2).collect()

[(4, (5, 199)), (2, (3, -99)), (2, (1, -99)), (19, (None, 23))]

In [66]:
x = sc.parallelize( [ ("NY", 10), ("OH", 20), ("OH", 99), ("CO", 88) ] )
y = sc.parallelize( [ ("NY", 30), ("CO", 40), ("NY", 22 )] )

In [67]:
x.join(y).collect()

[('NY', (10, 30)), ('NY', (10, 22)), ('CO', (88, 40))]

In [68]:
x.join(y).flatMap(lambda kv: ((kv[0],x) for x in kv[1])).collect()

[('NY', 10), ('NY', 30), ('NY', 10), ('NY', 22), ('CO', 88), ('CO', 40)]

In [69]:
x.cogroup(y).collect()

[('OH',
  (<pyspark.resultiterable.ResultIterable at 0x7fc7a7f95090>,
   <pyspark.resultiterable.ResultIterable at 0x7fc7a7f95650>)),
 ('NY',
  (<pyspark.resultiterable.ResultIterable at 0x7fc7a7f95510>,
   <pyspark.resultiterable.ResultIterable at 0x7fc7a7f95610>)),
 ('CO',
  (<pyspark.resultiterable.ResultIterable at 0x7fc7a809fbd0>,
   <pyspark.resultiterable.ResultIterable at 0x7fc7a809f790>))]

In [70]:
x.cogroup(y).flatMap(lambda kv: ((kv[0],x,y) for x in kv[1][0] for y in kv[1][1])).collect()

[('NY', 10, 30), ('NY', 10, 22), ('CO', 88, 40)]